In [68]:
import sys

import math
import time
from time import sleep
import googletrans
from googletrans import Translator
from langdetect import detect
import string
import numpy
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [69]:
# Loading 911 Call Dataset
## For allocation
# data = pd.read_csv("data.csv")

# For reallocation
data = pd.read_csv("allocated.csv")

# Loading Hospital Location Data 
hosp = pd.read_csv("hospital.csv")
hosp.head(n=2)

,no,name,address,latitude,longitude
0,1,Einstein Medical Center Montgomery,"559 W Germantown Pike, East Norriton, PA 19403...",40.153197,-75.343953
1,2,Suburban Community Hospital | Norristown,"2701 Dekalb Pike, Norristown, PA 19401, United...",40.140512,-75.315898


In [82]:
def hs_dist(lat1, lon1, lat2, lon2): 
      
    # distance formula 
    dlat = (lat2 - lat1) * math.pi / 180.0
    dlon = (lon2 - lon1) * math.pi / 180.0
  
    # convert to radians 
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
  
    # formulae 
    a = (pow(math.sin(dlat / 2), 2) + 
         pow(math.sin(dlon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2)); 
    rad = 6371
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

In [83]:
print(type(data.loc[2][3]))

print(hosp["longitude"].count())
print(hosp.iloc[33])

<class 'numpy.float64'>
34
no                                                          34
name                       Tower Health Chestnut Hill Hospital
address      8835 Germantown Ave, Philadelphia, PA 19118, U...
latitude                                               40.0798
longitude                                             -75.2123
Name: 33, dtype: object


In [84]:
def allot(lat, lon):
    
        min_dist = hs_dist(lat, lon, hosp.loc[0][3], hosp.loc[0][4])
        min_index = 1
        for i in range(1,hosp["longitude"].count()):
                dist = hs_dist(lat, lon, hosp.loc[i][3], hosp.loc[i][4])
                if(dist < min_dist):
                        min_dist = dist
                        min_index = i
        return hosp.iloc[min_index], min_dist

In [86]:
data['hospital_name'] = data['hospital_name'].astype(str)


In [87]:
def reallot(x,y):
    hosp.loc[34] = ['34', "New Hospital", "Address Not Defined", x, y]
    for i in range(1,data["lat"].count()):
        dist = hs_dist(x,y,data.loc[i][0],data.loc[i][1])
        if dist<data.loc[i][14]:       
            data.at[i]['hospital_no'] = hosp.loc[34][0]
            data.at[i]['hospital_name'] = hosp.loc[34][1]
            data.at[i]['hospital_lat'] = hosp.loc[34][3]
            data.at[i]['hospital_lon'] = hosp.loc[34][4]
            data.at[i]['distance'] = dist

In [88]:
alloted_hospital = allot(data.loc[2][0],data.loc[2][1])
print(alloted_hospital[0][2])
print(alloted_hospital[1])

Green St, Norristown, PA 19401, United States
1.2368780272494238


In [89]:
data.at[3,'lat'] = 0
data.head(n=5)

,lat,long,desc,zip,title,time,twp,addr,e,Category,hospital_no,hospital_name,hospital_lat,hospital_lon,distance
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,10-12-2015 17:10,NEW HANOVER,REINDEER CT & DEAD END,1,EMS,30.0,Phoenixville Hospital: Tullman Steve MD,40.123665,-75.515387,20.163664
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,10-12-2015 17:29,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS,27.0,Abington Hospital,40.251158,-75.268506,0.833485
2,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,10-12-2015 16:47,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS,13.0,Montco Human Services Center,40.123577,-75.332681,1.236878
3,0.000000,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,10-12-2015 16:56,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,EMS,20.0,Riddle Hospital,39.912754,-75.430919,4438.130196
4,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,10-12-2015 15:39,LANSDALE,CANNON AVE & W 9TH ST,1,EMS,4.0,Abington - Lansdale Hospital - Jefferson Health,40.250536,-75.271443,1.053476


In [90]:
data.to_csv('alloted.csv')